In [24]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('/home/gafur/Documents/Finance/Zillow_NLP/data/processed_zillow_reviews.csv') 

#I'll sample proportionally based on existing data distribution
target_size = 3000

sampled_df = df.groupby('Rating').apply(
    lambda x: x.sample(frac=target_size / len(df), random_state=42)
).reset_index(drop=True)

print(sampled_df['Rating'].value_counts())
sampled_df

Rating
5    2977
4      16
1       6
2       1
3       1
Name: count, dtype: int64


/tmp/ipykernel_48198/4102486677.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('Rating').apply(


,Rating,Address,ReviewDate,Description,Cleaned_Description
0,1,"7891 Lion St, Rancho Cucamonga, CA 91730",2018-11-29,"The experience with this agent is terrible. I do not feel like she represents me. All she represent is herself who wants a quick commission without adequate effort. Her knowledge, expertise, negotiation skills are so poor. There was a property line dispute but she told me no any problem in that property. After escrow was closed and the neighbor approached me and we found out. But even at that point, she just wanted to protect herself without providing any help to us. She never felt any sorry. Instead she pointed the finger to me and said I should review documents or contact the related persons by myself. In fact, I spent lots of hours to work on the problem. I do not know since I did anything, why I needed her at the beginning. I won't recommend anybody to her.",experience agent terrible feel like represent represent want quick commission adequate effort knowledge expertise negotiation skill poor property line dispute tell problem property escrow close neighbor approach find point want protect provide help feel sorry instead point finger say review document contact related person fact spend lot hour work problem know need beginning will recommend anybody
1,1,"840 E Green St #224 Grant Park - Pasadena, CA 91101",2018-09-26,Had a horrible experience working with Brian. He didn't have any insight into the market and the offer that he directed us to put on the property that we were interested in was 25 K above what the property eventually sold for. He cut down on contingency terms and also was not able to negotiate after inspection was done. He just didn't care and wanted to close a deal. Never work with him.,horrible experience work brian insight market offer direct property interested 25 k property eventually sell cut contingency term able negotiate inspection care want close deal work
2,1,"4817 Parker Ave, Sacramento, CA 95820",2020-07-02,"Very personable and bubbly. Unfortunately, when I hired her to help me buy my first home, she did not competently represent me by failing to relay information, forgetting to have me sign forms and not following my ask to request a seller credit for structural issues uncovered in the inspection. Great attitude and spirit but unfortunately not the Calibur of agent a first time buyer needs to help navigate a very complex and overwhelming process successfully.",personable bubbly unfortunately hire help buy home competently represent fail relay information forget sign form follow ask request seller credit structural issue uncover inspection great attitude spirit unfortunately calibur agent time buyer need help navigate complex overwhelming process successfully
3,1,"2266 Caminito Pescado UNIT 26, San Diego, CA 92107",2016-12-19,"Natalie convinced my husband and I to allow her to present our ""offer"" to the owners of a home she had listed. Against our better judgment we allowed her to represent us, as well as the seller. She was unprofessional and dishonest in the handling of our transaction. She insisted that she was ""doing everything she could"" to ""make it work."" The transaction (thankfully) did not go through, as we would have likely been severely further prejudiced by having Natalie represent us. Thankfully we found an honest and amazing agent to work with after this terrible experience!",natalie convince husband allow present offer owner home list well judgment allow represent seller unprofessional dishonest handling transaction insist work transaction thankfully likely severely prejudice have natalie represent thankfully find honest amazing agent work terrible experience
4,1,"659 Weng Star St, Beaumont, CA 92223",2016-01-05,After we decided on the home we wanted to purchase she completely pushed us to the side. As first time home buyers we felt neglected and uncared for. She was always too busy with other clients. Difficult to contact and took days to respond back even tho

In [25]:
#Use spaCy to pull out noun chunks:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_noun_chunks(text):
    doc = nlp(text)
    return [chunk.text.lower() for chunk in doc.noun_chunks]

sampled_df['Aspects'] = sampled_df['Description'].apply(extract_noun_chunks)
sampled_df

,Rating,Address,ReviewDate,Description,Cleaned_Description,Aspects
0,1,"7891 Lion St, Rancho Cucamonga, CA 91730",2018-11-29,"The experience with this agent is terrible. I do not feel like she represents me. All she represent is herself who wants a quick commission without adequate effort. Her knowledge, expertise, negotiation skills are so poor. There was a property line dispute but she told me no any problem in that property. After escrow was closed and the neighbor approached me and we found out. But even at that point, she just wanted to protect herself without providing any help to us. She never felt any sorry. Instead she pointed the finger to me and said I should review documents or contact the related persons by myself. In fact, I spent lots of hours to work on the problem. I do not know since I did anything, why I needed her at the beginning. I won't recommend anybody to her.",experience agent terrible feel like represent represent want quick commission adequate effort knowledge expertise negotiation skill poor property line dispute tell problem property escrow close neighbor approach find point want protect provide help feel sorry instead point finger say review document contact related person fact spend lot hour work problem know need beginning will recommend anybody,"[the experience, this agent, i, she, me, all, she, herself, who, a quick commission, adequate effort, her knowledge, expertise, negotiation skills, a property line dispute, she, me, no any problem, that property, escrow, the neighbor, me, we, that point, she, herself, any help, us, she, she, the finger, me, i, documents, the related persons, myself, fact, i, lots, hours, the problem, i, i, anything, i, her, the beginning, i, anybody, her]"
1,1,"840 E Green St #224 Grant Park - Pasadena, CA 91101",2018-09-26,Had a horrible experience working with Brian. He didn't have any insight into the market and the offer that he directed us to put on the property that we were interested in was 25 K above what the property eventually sold for. He cut down on contingency terms and also was not able to negotiate after inspection was done. He just didn't care and wanted to close a deal. Never work with him.,horrible experience work brian insight market offer direct property interested 25 k property eventually sell cut contingency term able negotiate inspection care want close deal work,"[a horrible experience, brian, he, any insight, the market, the offer, that, he, us, the property, that, we, what, the property, he, contingency terms, inspection, he, a deal, him]"
2,1,"4817 Parker Ave, Sacramento, CA 95820",2020-07-02,"Very personable and bubbly. Unfortunately, when I hired her to help me buy my first home, she did not competently represent me by failing to relay information, forgetting to have me sign forms and not following my ask to request a seller credit for structural issues uncovered in the inspection. Great attitude and spirit but unfortunately not the Calibur of agent a first time buyer needs to help navigate a very complex and overwhelming process successfully.",personable bubbly unfortunately hire help buy home competently represent fail relay information forget sign form follow ask request seller credit structural issue uncover inspection great attitude spirit unfortunately calibur agent time buyer need help navigate complex overwhelming process successfully,"[bubbly, i, her, me, my first home, she, me, information, me, forms, my ask, a seller credit, structural issues, the inspection, great attitude, spirit, unfortunately not the calibur, agent, a first time buyer, a very complex and overwhelming process]"
3,1,"2266 Caminito Pescado UNIT 26, San Diego, CA 92107",2016-12-19,"Natalie convinced my husband and I to allow her to present our ""offer"" to the owners of a home she had listed. Against our better judgment we allowed her to represent us, as well as the seller. She was unprofessional and dishonest in the handling of our transaction. She insisted that

In [12]:
sampled_df

,Rating,Address,ReviewDate,Description,Cleaned_Description,Aspects,Cleaned_Aspects
0,1,"7891 Lion St, Rancho Cucamonga, CA 91730",2018-11-29,"The experience with this agent is terrible. I do not feel like she represents me. All she represent is herself who wants a quick commission without adequate effort. Her knowledge, expertise, negotiation skills are so poor. There was a property line dispute but she told me no any problem in that property. After escrow was closed and the neighbor approached me and we found out. But even at that point, she just wanted to protect herself without providing any help to us. She never felt any sorry. Instead she pointed the finger to me and said I should review documents or contact the related persons by myself. In fact, I spent lots of hours to work on the problem. I do not know since I did anything, why I needed her at the beginning. I won't recommend anybody to her.",experience agent terrible feel like represent represent want quick commission adequate effort knowledge expertise negotiation skill poor property line dispute tell problem property escrow close neighbor approach find point want protect provide help feel sorry instead point finger say review document contact related person fact spend lot hour work problem know need beginning will recommend anybody,"[the experience, this agent, i, she, me, all, she, herself, who, a quick commission, adequate effort, her knowledge, expertise, negotiation skills, a property line dispute, she, me, no any problem, that property, escrow, the neighbor, me, we, that point, she, herself, any help, us, she, she, the finger, me, i, documents, the related persons, myself, fact, i, lots, hours, the problem, i, i, anything, i, her, the beginning, i, anybody, her]","[experience, agent, quick commission, adequate effort, knowledge, expertise, negotiation skills, property line dispute, problem, property, escrow, neighbor, point, help, finger, documents, related persons, fact, lots, hours, problem, beginning, anybody]"
1,1,"840 E Green St #224 Grant Park - Pasadena, CA 91101",2018-09-26,Had a horrible experience working with Brian. He didn't have any insight into the market and the offer that he directed us to put on the property that we were interested in was 25 K above what the property eventually sold for. He cut down on contingency terms and also was not able to negotiate after inspection was done. He just didn't care and wanted to close a deal. Never work with him.,horrible experience work brian insight market offer direct property interested 25 k property eventually sell cut contingency term able negotiate inspection care want close deal work,"[a horrible experience, brian, he, any insight, the market, the offer, that, he, us, the property, that, we, what, the property, he, contingency terms, inspection, he, a deal, him]","[horrible experience, brian, insight, market, offer, property, property, contingency terms, inspection, deal]"
2,1,"4817 Parker Ave, Sacramento, CA 95820",2020-07-02,"Very personable and bubbly. Unfortunately, when I hired her to help me buy my first home, she did not competently represent me by failing to relay information, forgetting to have me sign forms and not following my ask to request a seller credit for structural issues uncovered in the inspection. Great attitude and spirit but unfortunately not the Calibur of agent a first time buyer needs to help navigate a very complex and overwhelming process successfully.",personable bubbly unfortunately hire help buy home competently represent fail relay information forget sign form follow ask request seller credit structural issue uncover inspection great attitude spirit unfortunately calibur agent time buyer need help navigate complex overwhelming process successfully,"[bubbly, i, her, me, my first home, she, me, information, me, forms, my ask, a seller credit, structural issues, the inspection, great attitude, spirit, unfortunately not the calibur, agent, a first time buyer, a very comple

In [22]:
from spacy.lang.en.stop_words import STOP_WORDS

def clean_aspects(aspect_list):
    cleaned = []
    for phrase in aspect_list:
        words = phrase.split()
        
        # Remove leading stopwords
        while words and words[0] in STOP_WORDS:
            words = words[1:]
        
        # Remove trailing stopwords
        while words and words[-1] in STOP_WORDS:
            words = words[:-1]
        
        # Join back if anything left and longer than 1 char
        if words:
            new_phrase = ' '.join(words).strip()
            if len(new_phrase) > 1:
                cleaned.append(new_phrase)
    return cleaned


sampled_df['Cleaned_Aspects'] = sampled_df['Aspects'].apply(clean_aspects)


In [23]:
from collections import Counter

all_aspects = [aspect for aspects in sampled_df['Aspects'] for aspect in aspects]
aspect_counts = Counter(all_aspects)

# Show top 20
print(aspect_counts.most_common(20))


[('we', 5821), ('i', 5189), ('she', 4192), ('us', 3716), ('he', 2963), ('you', 1853), ('me', 1653), ('it', 1553), ('that', 1273), ('her', 1178), ('him', 809), ('what', 765), ('a home', 759), ('the process', 754), ('they', 526), ('anyone', 493), ('who', 486), ('everything', 441), ('all', 389), ('which', 346)]
